# Dance Gestures (Drafted)

Gestures in dance refer to movements made by the dancer to convey a particular meaning or feeling. They can be subtle or overt and may include movements of the hands, arms, head, and body. Gestures can be used to express emotions, tell stories, and enhance the overall expressive power of the dance.

In this notebook, we are examining [Emma Joubert's Skinny Love (Birdy cover) Contemporary Solo](https://www.youtube.com/watch?v=QOlSCBRmfWY) and using [Mediapipe](https://google.github.io/mediapipe/) to extract x,y,z landmarks using its Pose Estimation Model. Numerical approach will be used to detect classic dance movements. But for now, only basic poses will be used.

## Import Module

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
import cv2

In [7]:
cap = cv2.VideoCapture('./data/interim/frames/contemporary_dance_solo_20230219225812_10_annotated.mp4')
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(f"h: {h}, w: {w}")
cap.release()
df = pd.read_csv('./data/interim/landmarks/contemporary_dance_solo_20230219225812_10.csv')
df.set_index('frame', inplace=True)

h: 360, w: 640


In [8]:
print(df.columns)

Index(['nose_x', 'nose_y', 'nose_z', 'left_eye_inner_x', 'left_eye_inner_y',
       'left_eye_inner_z', 'left_eye_x', 'left_eye_y', 'left_eye_z',
       'left_eye_outer_x', 'left_eye_outer_y', 'left_eye_outer_z',
       'right_eye_inner_x', 'right_eye_inner_y', 'right_eye_inner_z',
       'right_eye_x', 'right_eye_y', 'right_eye_z', 'right_eye_outer_x',
       'right_eye_outer_y', 'right_eye_outer_z', 'left_ear_x', 'left_ear_y',
       'left_ear_z', 'right_ear_x', 'right_ear_y', 'right_ear_z',
       'mouth_left_x', 'mouth_left_y', 'mouth_left_z', 'mouth_right_x',
       'mouth_right_y', 'mouth_right_z', 'left_shoulder_x', 'left_shoulder_y',
       'left_shoulder_z', 'right_shoulder_x', 'right_shoulder_y',
       'right_shoulder_z', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z',
       'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'left_wrist_x',
       'left_wrist_y', 'left_wrist_z', 'right_wrist_x', 'right_wrist_y',
       'right_wrist_z', 'left_pinky_x', 'left_pinky_y', 'left_

In [3]:
df.loc[:, df.columns.str.endswith('_x')] *= w
df.loc[:, df.columns.str.endswith('_y')] *= h
df.loc[:, df.columns.str.endswith('_y')] = h - df.loc[:, df.columns.str.endswith('_y')]

# Understanding the Z-coordinate can help us determine the quality of the dance video
# If the mean average is close to 0, it may indicate the subject's position is consistent
z_cols = df.filter(regex='_z$')
z_min = round(z_cols.min().min(),2)
z_max = round(z_cols.max().max(),2)
z_mean = round(z_cols.mean().mean())
print(f"Smallest z-value: {z_min}, Largest z-value: {z_max}, Average: {z_mean}")

Smallest z-value: -0.66, Largest z-value: 0.71, Average: 0


### Mediapipe's Post Landmark
![Image](https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png)

### Defining Basic Poses

![Image](https://i.ibb.co/zFDftG3/poses.png)

Let's first define the basic positions using using a numerical approach

1. `Standing`: If the difference between the height of the right and left sides is less than 0.08

2. `Sitting down`: If the difference between the y-axis 

3. `Bent Knee`: If the angle from hip to knee not within the range of 85-95

## Numerical Operations
Numerical operations required to get joint estimates, distances, and angles using geometry

In [4]:
# Estimated forehead position
df.loc[:, 'forehead_x'] = (df['left_eye_outer_x'] + df['right_eye_outer_x']) / 2
df.loc[:, 'forehead_y'] = (df['left_eye_outer_y'] + df['right_eye_outer_y']) / 2
df.loc[:, 'forehead_z'] = (df['left_eye_outer_z'] + df['right_eye_outer_z']) / 2

# Estimated torso position
df.loc[:, 'torso_x'] = (df['left_shoulder_x'] + df['right_shoulder_x'] + df['left_hip_x'] + df['right_hip_x']) / 4
df.loc[:, 'torso_y'] = (df['left_shoulder_y'] + df['right_shoulder_y'] + df['left_hip_y'] + df['right_hip_y']) / 4
df.loc[:, 'torso_z'] = (df['left_shoulder_z'] + df['right_shoulder_z'] + df['left_hip_z'] + df['right_hip_z']) / 4

# Eucleadian distance between right shoulder to hip
df.loc[:, 'dis_right_shoulder_to_hip'] = np.sqrt((df['right_hip_y'] - df['right_knee_y'])**2)
df.loc[:, 'dis_left_shoulder_to_hip'] = np.sqrt((df['left_hip_y'] - df['left_knee_y'])**2)

# Eucleadian distance between hip to knee
df.loc[:, 'dis_right_hip_to_knee'] = np.sqrt((df['right_hip_y'] - df['right_knee_y'])**2)
df.loc[:, 'dis_left_hip_to_knee'] = np.sqrt((df['left_hip_y'] - df['left_knee_y'])**2)

# Eucleadian distance between knee to ankle
df.loc[:, 'dis_right_knee_to_ankle'] = np.sqrt((df['right_knee_y'] - df['right_ankle_y'])**2)
df.loc[:, 'dis_left_knee_to_ankle'] = np.sqrt((df['left_knee_y'] - df['left_ankle_y'])**2)

# Eucleadian distance between forehead to right shoulder
df.loc[:, 'dis_forehead_to_right_shoulder'] = np.sqrt((df['forehead_y'] - df['right_shoulder_y'])**2)
df.loc[:, 'dis_forehead_to_left_shoulder'] = np.sqrt((df['forehead_y'] - df['left_shoulder_y'])**2)

# Difference between height to the left and to the right
df.loc[:, 'total_height_r'] = df['dis_forehead_to_right_shoulder'] + df['dis_right_shoulder_to_hip'] + df['dis_right_hip_to_knee'] + df['dis_right_knee_to_ankle']
df.loc[:, 'total_height_l'] = df['dis_forehead_to_left_shoulder'] + df['dis_left_shoulder_to_hip'] + df['dis_left_hip_to_knee'] + df['dis_left_knee_to_ankle']
df.loc[:,'diff_height_r_and_l'] = (df['total_height_r'] - df['total_height_l']).abs()

# Difference in x-axis from hip to ankle
df.loc[:, 'diff_hip_to_ankle_r'] = (df['right_hip_x']-df['right_ankle_x']).abs()
df.loc[:, 'diff_hip_to_ankle_l'] = (df['left_hip_x']-df['left_ankle_x']).abs()

# calculate the angle between the hip and knee joint for the right leg
df['angle_hip_knee_r'] = df.apply(lambda row: math.degrees(math.atan2(row['right_knee_y'] - row['right_hip_y'], row['right_knee_x'] - row['right_hip_x'])), axis=1)
df['angle_hip_knee_l'] = df.apply(lambda row: math.degrees(math.atan2(row['left_knee_y'] - row['left_hip_y'], row['left_knee_x'] - row['left_hip_x'])), axis=1)

## Basic Poses Algorithms
This code estimates positions of different body parts, computes Euclidean distances between them, calculates the difference between heights, difference in x-axis from hip to ankle, and the angle between the hip and knee joint for the left and right legs for each row in the dataframe.

In [5]:
def measure_pose_standing(df, threshold):
    dfs = df.copy()
    dfs['diff_rate_r'] = dfs['diff_height_r_and_l'] / dfs['total_height_r']
    dfs['diff_rate_l'] = dfs['diff_height_r_and_l'] / dfs['total_height_l']
    dfs['both_legs_straight'] = (dfs['diff_rate_r'] <= threshold) & (dfs['diff_rate_l'] <= threshold)
    total = dfs['both_legs_straight'].sum()
    return round(total/24.9,2)

print(f"Estimated standing position {measure_pose_standing(df,0.08)} s")

def measure_pose_sitting(df, threshold):
    dfs = df.copy()
    dfs['diff_rate_r'] = dfs['diff_hip_to_ankle_r']/((dfs['right_hip_x']+dfs['right_ankle_x'])/2)
    dfs['diff_rate_l'] = dfs['diff_hip_to_ankle_r']/((dfs['right_hip_x']+dfs['right_ankle_x'])/2)
    dfs['both_legs_sitting'] = (dfs['diff_rate_r'] <= threshold) & (dfs['diff_rate_l'] <= threshold)
    total = dfs['both_legs_sitting'].sum()
    return round(total/24.9,2)

print(f"Estimated sitting position {measure_pose_sitting(df,0.08)} s")


def measure_pose_bent_knees(df):
    dfs = df[['angle_hip_knee_r','angle_hip_knee_l']].abs()
    dfs['bent_knees'] = (dfs['angle_hip_knee_r'] < 80) | (dfs['angle_hip_knee_r'] > 100) & (dfs['angle_hip_knee_l'] < 80) | (dfs['angle_hip_knee_l'] > 100)
    total = dfs['bent_knees'].sum()
    return round(total/24.9,2)

print(f"Estimated bent knee position {measure_pose_bent_knees(df)} s")

Estimated standing position 23.53 s
Estimated sitting position 37.55 s
Estimated bent knee position 59.36 s


## To be continued